# Job Description-Based Resume Aligning Crew

## Initialization & Setup

In [1]:
import yaml
import json
from pathlib import Path
from pydantic import BaseModel
from crewai import Agent, Task, Crew, LLM

d:\TechStuff\ai-garage\.venv\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# llm = LLM(
#     model="ollama/llama3.2:1b",
#     base_url="http://localhost:11434"
# )

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

llm = LLM(
    model=os.getenv("AZURE_MODEL")
)

## Resume Aligner Crew

In [4]:
# Load agent and task configurations from YAML files
with open('resume_optimizer_crew/config/agents.yaml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('resume_optimizer_crew/config/tasks.yaml', 'r') as f:
    tasks_config = yaml.safe_load(f)

In [5]:
class ResumeAnalyis(BaseModel):
    strengths: list[str]
    areas_of_improvement: list[str]
    recommendations: list[str]
    score: int

class JobDescriptionAnalysis(BaseModel):
    objectives_and_expectations: list[str]
    core_competencies: list[str]
    ats_friendly_keywords: list[str]

class OptimizedResume(BaseModel):
    resume_content: str

### Define Custom Tool to Read Markdown-Formatted Resumes

The `FileReadTool` from `crewai_tools` throws errors due to invalid character encodings present in most markdown files, so we create a custom tool to read from the markdown files using UTF-8 encoding

In [6]:
from crewai.tools import BaseTool

class ResumeMarkdownFileReadTool(BaseTool):
    name: str = "resume_markdown_file_read_tool"
    description: str = "Reads the contents of a resume stored in a markdown file, using the path provided (in UTF-8 format)."

    def _run(self, resume_file_path: str) -> str:
        with open(resume_file_path, 'r', encoding="utf-8") as f:
            resume_content = f.read()
        
        return resume_content
    
resume_read_tool = ResumeMarkdownFileReadTool()


### Define the Agents, Tasks & Crews based on YAML Configs

In [7]:
resume_analyzer = Agent(
    config=agents_config["resume_analyzer"],
    tools=[resume_read_tool],
    llm=llm
)

job_description_analyzer = Agent(
    config=agents_config["job_description_analyzer"],
    llm=llm
)

alignment_specialist = Agent(   
    config=agents_config["alignment_specialist"],
    tools=[resume_read_tool],
    llm=llm
)

In [8]:
analyze_resume = Task(
    config=tasks_config["analyze_resume"],
    agent=resume_analyzer,
    output_pydantic=ResumeAnalyis
)

analyze_job_description = Task(
    config=tasks_config["analyze_job_description"],
    agent=job_description_analyzer,
    output_pydantic=JobDescriptionAnalysis
)

fine_tune_resume = Task(
    config=tasks_config["fine_tune_resume"],
    agent=alignment_specialist,
    output_pydantic=OptimizedResume
)

In [16]:
resume_analysis_crew = Crew(
    agents=[resume_analyzer],
    tasks=[analyze_resume],
    verbose=False
)

resume_optimization_crew = Crew(
    agents=[job_description_analyzer, alignment_specialist],
    tasks=[analyze_job_description, fine_tune_resume],
    verbose=False
)

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


### Test the Crews

In [17]:
resume_analysis_result = resume_analysis_crew.kickoff(
    inputs = {
        "resume_path": "sample_resume/JonathanWhitmore_DataScience/resume_original.md"
    }
)


In [18]:
resume_analysis_result.pydantic.model_dump()

{'strengths': ['Extensive experience in data science and data engineering.',
  'Strong educational background with a PhD in Physics.',
  'Proficient in multiple programming languages including Python, SQL, and R.',
  'Experience with data visualization and statistical modeling.',
  'Involvement in various data science projects and consulting roles.',
  'Good understanding of both theoretical and practical aspects of data science.',
  'Experience in publishing, speaking, and contributing to open source projects.',
  'Relevant skills in tools such as Jupyter Notebook, pandas, matplotlib, numpy, scikit-learn, etc.'],
 'areas_of_improvement': ['The contact information is cluttered and contains unnecessary placeholders.',
  'The structure of the resume could be better organized for readability.',
  'Inconsistent use of bullet points and formatting.',
  'Lack of clear quantifiable achievements in job descriptions.'],
 'recommendations': ['Remove placeholders and unnecessary icons from the co

In [20]:
resume_optimization_result = resume_optimization_crew.kickoff(
    inputs = {
        "resume_path": "sample_resume/JonathanWhitmore_DataScience/resume_original.md",
        "job_description": """Day to Day

At Indeed, our mission is to Help People Get Jobs. Our products help transform the lives of millions of people by connecting them with meaningful employment through the power of AI.
As a Senior Data Scientist, you will play a critical role in advancing this mission by leveraging the latest AI developments to drive innovation.
In this role, you will assess data and formulate problem statements and associated opportunity sizes. Your contributions will enhance our platform’s application quality capabilities using AI. You will explore the use of AI models to assess job seekers' qualifications for a job while also supporting initiatives like improving the understanding of candidate communication in key Markets. Additionally, you will be driving innovation by using GenAI techniques to assist with applications while also providing AI-powered candidate evaluations to employers.

Your work will also involve building and refining automated model retraining pipelines, ensuring that our models stay responsive and effective in the face of evolving data. By utilizing advanced A/B testing frameworks and collaborating across teams, you will continuously assess and optimize the performance of these models. Your efforts will help build smarter systems that directly contribute to the success of millions of job seekers globally.
These are just a few examples of the impactful work you will drive in this role. The opportunities are countless.

Responsibilities

Develop machine learning models from prototype to production
Advocate and establish best practices in AI/ML system design, data pipeline architecture, and large-scale deployment.
Collaborate cross-functionally with teams across the company on model inference and product applications
Break down larger Machine Learning initiatives into pieces that deliver incremental business value and implement them
Deeply understand and optimize large-scale distributed AI systems to improve performance and efficiency.
Drive the AI/ML strategy by identifying business opportunities and aligning machine learning initiatives with company goals.
Influence and guide cross-functional teams in the application of AI/ML to solve key business challenges.
Balance technical and product thinking, and show ownership for team goals and Indeed’s mission to help people get jobs
Experiment with Proof of Concept Machine Learning models, scale them to production and run iterative A/B experiments to improve our technology
Skills/Competencies

We are looking for someone with a deep interest in designing and building AI/ML systems, loves thought-provoking assignments, is eager to learn, and wants to work with a variety of technologies to “help people get jobs”.

Bachelors in Computer Science, Mathematics, Statistics, or related fields
3+ years of industry experience as a Software Engineer, Data Scientist, or Machine Learning Engineer
Proven track record of successfully delivering high-impact AI solutions, with a focus on production deployment and operationalization (MLOps).
Proven interpersonal and collaboration skills, with experience working alongside product managers, engineers, and executive leaders.
Solid knowledge of data structures and algorithms
Experience examining and running A/B tests, ability to understand and make business tradeoffs
Experience or exposure to training, fine-tuning, and evaluating language models
Solid expertise in deep learning frameworks (e.g., TensorFlow, PyTorch), language models, reinforcement learning, and generative AI.
""",
    "recommendations": "- " + "\n- ".join(resume_analysis_result.pydantic.recommendations)
    }
)

In [21]:
print(resume_optimization_result.pydantic.resume_content)

# Jonathan Whitmore, PhD

**Data Scientist | AI/ML Expert**

## Contact Information

- Location: Mountain View, CA
- Phone: +1 650-943-3715
- Email: JBWhit@gmail.com
- Portfolio: [JonathanWhitmore.com](http://JonathanWhitmore.com)
- LinkedIn: [JonathanBWhitmore](https://www.linkedin.com/in/JonathanBWhitmore)
- GitHub: [JBWhit](https://github.com/JBWhit)

## Summary

PhD-educated Data Scientist with extensive experience in AI and Machine Learning, focusing on developing, deploying, and optimizing large-scale distributed AI systems. Proven ability to identify business opportunities and create value through data science and AI/ML innovation. Strong background in data engineering, statistical modeling, and cross-functional collaboration. Passionate about driving AI advancements to enhance product quality and operational efficiency.

## Professional Experience

### Data Scientist
**Silicon Valley Data Science**, Mountain View, CA | 2014 - Present

- Led consulting projects for multiple clie

## Convert Resume to Markdown Format

In [19]:
import os

resumes = [
    os.path.join(".", "sample_resume", file) for file in os.listdir(os.path.join(".", "sample_resume"))
]

print(resumes)


['.\\sample_resume\\ChloeMartinez_SeniorProductManager.pdf', '.\\sample_resume\\JasmineBell_UXDesigner.pdf', '.\\sample_resume\\JohnGonalez_PythonDeveloper.docx', '.\\sample_resume\\JohnSmith_SalesExecutive.pdf', '.\\sample_resume\\JonathanWhitmore_DataScience.pdf']


### Experiment with [Docling](https://ds4sd.github.io/docling/)

In [1]:
from docling.document_converter import DocumentConverter

d:\TechStuff\ai-garage\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
converter = DocumentConverter()
result = converter.convert(resumes[0])

print(result.document.export_to_markdown())

d:\TechStuff\ai-garage\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tezan\.cache\huggingface\hub\models--ds4sd--docling-models. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


## Jonathan Whitmore

PhD, Data Scientist

## Experience

2014-

Present

2014

Insight Fellow , Insight Data Science , Palo Alto, CA, USA.

2011-2014

Postdoctoral Research Associate , Swinburne University , Melbourne, AUS.

2005-2011

Graduate Student Researcher , UCSD , San Diego, CA, USA.

- { Developed a novel technique to extract information from high resolution spectroscopic data that led to uncovering unknown short-range systematic errors.

## Programming and Development Skills

Languages

Python, SQL (Impala/Hive), R, L A T X, shell scripts, CSS, HTML.

E

Tools

Jupyter Notebook, pandas, matplotlib, numpy, scikit-learn, scipy, pymc3, git, pandoc.

## Publishing, Speaking, and Side Projects

2016 O'Reilly author: Jupyter Notebook for Data Science Teams [screencast], editor O'Reilly Media.

2016 UC Berkeley Guest Lecturer: Master in Data Science lecture on Jupyter Notebook.

2015 Open Source Speaker: OSCON.

2014-2015 Technical reviewer of Mastering SciPy by Francisco J. Blanco

### Experiment with [MarkItDown](https://github.com/microsoft/markitdown)

In [20]:
from markitdown import MarkItDown

In [5]:
md = MarkItDown()
result = md.convert(resumes[0])
print(result.text_content)

Jonathan Whitmore
PhD, Data Scientist

Experience

Mountain View, CA
+1 650-943-3715
(cid:66) JBWhit@gmail.com
(cid:205) JonathanWhitmore.com
JBWhit
JonathanBWhitmore

2014-
Present

Data Scientist, Silicon Valley Data Science, Mountain View, CA, USA.
(cid:123) Consulting as a member of several small data science/data engineering teams at multiple companies.
(cid:123) Creating output to explain data analysis, data visualization, and statistical modeling results to managers.
(cid:123) Modeling survey data responses with ordinal logistic regression in R.
(cid:123) Analyzing and visualizing user behavior migration.

2014 Insight Fellow, Insight Data Science, Palo Alto, CA, USA.

(cid:123) Created a Data Science project to predict the auction sale price of Abstract Expressionist art.

2011–2014 Postdoctoral Research Associate, Swinburne University, Melbourne, AUS.

(cid:123) Cleaned noisy and inhomogeneous astronomical data taken over four years by diﬀerent observing groups.
(cid:123) Cura

In [10]:
result = md.convert(resumes[1])
print(result.text_content)



![](data:image/png;base64...)JOHN **GONALEZ**

Python Developer

**CONTACT**

Pittsburgh, PA

1. 456-7890 john1652@gmail.com [LinkedIn](http://linkedin.com/in/john-g) [Github](http://github.com/john-gonzalez)

**EDUCATION**

**M.S.**

**Computer Science**

University of Chicago

Chicago, IL / 2014 - 2016

**B.S.**

**Computer Science**

University of Pittsburgh

Pittsburgh, PA / 2010 - 2014

**SKILLS**

HTML/ CSS

SQL (PostgreSQL, Oracle)

JavaScript (Angular)

Python (Django)

REST APIs (GraphQL)

AWS (Redshift, S3)

Git

**CAREER OBJECTIVE**

Experienced Python developer with extensive Django experience looking to continue to develop my skill set on the back-end at a company driven to addressing the climate crisis.

**WORK EXPERIENCE**

Python Developer

| DoorDash / September 2018 - current / Chicago IL | |
| --- | --- |
| · | Worked on building new Angular components for the customer- |
|  | facing web app which improved the time on page for the |
| · | average user by 2 minutes 

In [13]:
result = md.convert(resumes[2])
print(result.text_content)

CHLOE MARTINEZ
Senior Product Manager | Digital Advertising | Monetization Expert

E

 1  234  555 1234



help@enhancv.com

q

linkedin.com



Phoenix, Arizona

SUMMARY

With over 6 years of product management experience in ad tech, I bring a
wealth of knowledge in DSP/SSP platforms, predictive targeting, and advanced
modeling to drive product strategy and successful launch execution.

EXPERIENCE

Senior Product Manager, Adtech Solutions
Google

•

•

•

•

•

•

01/2018   12/2022
San Francisco, CA, USA
Led the development of a new programmatic advertising platform,
overseeing the project from conception to launch, achieving a 20% market
share within the first year.
Managed cross-functional teams to improve auction logic and yield
management, resulting in a 15% revenue increase for our top 100 clients.
Orchestrated the integration of machine learning algorithms for predictive
targeting, improving ad relevance by 25% and user engagement by 30%.
Drove the product roadmap and strategy 

### Comparison b/w Docling & MarkItDown

In [20]:
import time
import pandas as pd

pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',504)
pd.set_option('display.width',1000)

# Initialize a list to store the results
results = []

# Measure time taken for each conversion using docling
for resume in resumes:
    start_time = time.time()
    docling_result = converter.convert(resume)
    docling_time = time.time() - start_time
    results.append({
        'Document': resume,
        'Converter': 'docling',
        'Time Taken (s)': docling_time
    })

# Measure time taken for each conversion using markitdown
for resume in resumes:
    start_time = time.time()
    markitdown_result = md.convert(resume)
    markitdown_time = time.time() - start_time
    results.append({
        'Document': resume,
        'Converter': 'markitdown',
        'Time Taken (s)': markitdown_time
    })

# Create a DataFrame to present the results
df_results = pd.DataFrame(results)

# Display the results
print(df_results)
print("\n\n")

# Derive insights
insights = df_results.groupby('Converter')['Time Taken (s)'].describe()
print(insights)

d:\TechStuff\ai-garage\.venv\Lib\site-packages\docling\pipeline\standard_pdf_pipeline.py:189: DeprecationWarning: Field `generate_table_images` is deprecated. To obtain table images, set `PdfPipelineOptions.generate_page_images = True` before conversion and then use the `TableItem.get_image` function.
  or self.pipeline_options.generate_table_images
d:\TechStuff\ai-garage\.venv\Lib\site-packages\docling\pipeline\standard_pdf_pipeline.py:189: DeprecationWarning: Field `generate_table_images` is deprecated. To obtain table images, set `PdfPipelineOptions.generate_page_images = True` before conversion and then use the `TableItem.get_image` function.
  or self.pipeline_options.generate_table_images
d:\TechStuff\ai-garage\.venv\Lib\site-packages\docling\pipeline\standard_pdf_pipeline.py:189: DeprecationWarning: Field `generate_table_images` is deprecated. To obtain table images, set `PdfPipelineOptions.generate_page_images = True` before conversion and then use the `TableItem.get_image` fun

                                            Document   Converter  Time Taken (s)
0  .\sample_resume\ChloeMartinez_SeniorProductMan...     docling        2.014914
1         .\sample_resume\JasmineBell_UXDesigner.pdf     docling        1.199052
2   .\sample_resume\JohnGonalez_PythonDeveloper.docx     docling        0.096088
3       .\sample_resume\JohnSmith_SalesExecutive.pdf     docling        2.273818
4   .\sample_resume\JonathanWhitmore_DataScience.pdf     docling        1.037285
5  .\sample_resume\ChloeMartinez_SeniorProductMan...  markitdown        0.256180
6         .\sample_resume\JasmineBell_UXDesigner.pdf  markitdown        0.157676
7   .\sample_resume\JohnGonalez_PythonDeveloper.docx  markitdown        0.039202
8       .\sample_resume\JohnSmith_SalesExecutive.pdf  markitdown        0.130077
9   .\sample_resume\JonathanWhitmore_DataScience.pdf  markitdown        0.048968



            count      mean       std       min       25%       50%       75%       max
Converter         

## Job Description-Based Resume Optimization Flow

In [22]:
from markitdown import MarkItDown
from crewai.flow.flow import Flow, start, listen

In [23]:
class ResumeOptimizationState(BaseModel):
    job_description: str = ""
    resume_path: Path = "sample_resume/"
    resume_markdown_path: Path = "sample_resume/"
    resume_analysis_result: ResumeAnalyis = None
    resume_optimization_result: OptimizedResume = None


class ResumeOptimizationFlow(Flow[ResumeOptimizationState]):
    def __init__(self):
        super().__init__()
        self.md_converter = MarkItDown()


    @start()
    def convert_resume_to_markdown(self):
        print("Converting resume to markdown")
        if os.path.exists(self.state.resume_path):
            result = self.md_converter.convert(self.state.resume_path)

            file_name = os.path.splitext(os.path.basename(self.state.resume_path))[0]
            
            os.makedirs(f"sample_resume/{file_name}", exist_ok=True)

            self.state.resume_markdown_path = f"sample_resume/{file_name}/resume_original.md"

            with open(self.state.resume_markdown_path, "w", encoding="utf-8") as f:
                f.write(result.text_content)
            print("Resume converted to markdown\n")
        else:
            print("Resume path does not exist")
    
    @listen(convert_resume_to_markdown)
    def resume_analysis(self):
        print("Analyzing resume")
        resume_analysis_result = resume_analysis_crew.kickoff(
            inputs = {
                "resume_path": self.state.resume_markdown_path
            }
        )
        print("Resume analysis complete\n")
        self.state.resume_analysis_result = resume_analysis_result.pydantic

    @listen(resume_analysis)
    def save_resume_analysis(self):
        print("Saving resume analysis")
        file_name = os.path.splitext(os.path.basename(self.state.resume_path))[0]
        analysis_path = f"sample_resume/{file_name}/resume_analysis.json"
        with open(analysis_path, "w") as f:
            json.dump(self.state.resume_analysis_result.model_dump(), f, indent=4)
        print(f"Resume analysis saved at {analysis_path}\n")

    @listen(resume_analysis)
    def optimize_resume(self):
        print("Optimizing resume")
        resume_optimization_result = resume_optimization_crew.kickoff(
            inputs = {
                "resume_path": self.state.resume_markdown_path,
                "job_description": self.state.job_description,
                "recommendations": "- " + "\n- ".join(self.state.resume_analysis_result.recommendations)
            }
        )
        print("Resume optimization complete\n")
        self.state.resume_optimization_result = resume_optimization_result.pydantic

    @listen(optimize_resume)
    def save_optimized_resume(self):
        print("Saving optimized resume")
        file_name = os.path.splitext(os.path.basename(self.state.resume_path))[0]
        optimized_resume_path = f"sample_resume/{file_name}/resume_optimized.md"
        with open(optimized_resume_path, "w", encoding="utf-8") as f:
            f.write(self.state.resume_optimization_result.resume_content)
        print(f"Aligned resume saved at {optimized_resume_path}\n")

    
    


In [ ]:
flow = ResumeOptimizationFlow()
flow.plot()

Plot saved as crewai_flow.html


: 

In [22]:
flow.state.resume_path = "sample_resume/JonathanWhitmore_DataScience.pdf"

flow.state.job_description = """
Job Title : Lead Data Scientist

Location : Hyderabad

Company : Awone DataScience



Role Description:

Awone DataScience is seeking an experienced Lead Data Scientist to join our team. This is a full-time on-site opportunity designed for a senior-level professional with expertise in machine learning, AI, and advanced data science techniques. The ideal candidate will be responsible for developing cutting-edge data models, mentoring junior data scientists, and driving the integration of data science solutions into the company’s products. In this role, you will collaborate with cross-functional teams and communicate insights to both technical and non-technical stakeholders, ensuring that data-driven strategies align with the company's objectives.



Key Responsibilities:

Design and develop custom data models and algorithms tailored to complex data sets.
Lead the development and implementation of advanced machine learning models to solve intricate business problems.
Communicate data-driven insights, findings, and recommendations clearly to Product Managers and executives.
Mentor and guide a team of junior data scientists, fostering growth and expertise in best practices.
Collaborate with cross-functional teams to define and execute data-driven strategies and integrate solutions into the company’s products.
Continuously evaluate and refine the performance of existing machine learning models and algorithms.
Stay abreast of the latest trends and innovations in AI/ML, proposing new solutions to enhance the company’s capabilities.
Ensure end-to-end delivery of scalable, optimized, and enterprise-level AI solutions, including model deployment and monitoring.


Qualifications:

Education: Bachelor’s or Master’s degree in Computer Science, Engineering, or a related technical field.
Experience: A minimum of 5-8 years of experience in data science, with at least 2 years in a leadership role, managing teams of junior data scientists.
Proven experience in the end-to-end delivery of scalable, optimized, and enterprise-grade AI/ML solutions.
Expertise in fine-tuning and prompt engineering of AI/ML models, particularly in the context of Generative AI (GenAI), Large Language Models (LLMs), and Transformer-based architectures (e.g., GPT, BERT).
Hands-on experience in Natural Language Processing (NLP) tools such as Word2Vec, TextBlob, NLTK, SpaCy, Gensim, CoreNLP, BERT, GloVe, and related technologies.
Strong experience with cloud platforms such as AWS, Azure, or GCP, including model deployment using frameworks like FastAPI or gRPC.
Experience with containerization (Docker) and deploying ML models in Kubernetes clusters.
Proficiency in both NoSQL and SQL databases.
Strong programming skills in Python and familiarity with relevant data science libraries (e.g., NumPy, Pandas, Scikit-learn, TensorFlow, PyTorch).
Hands-on experience with big data technologies such as Hadoop, Spark, and Hive.
Expertise in statistical methods, experimental design, and hypothesis testing.
Familiarity with MLOps practices, including CI/CD pipelines for machine learning.
Strong project management skills, with the ability to juggle multiple projects and deadlines.
Excellent communication and presentation skills, with the ability to translate complex technical concepts for non-technical stakeholders.
Domain knowledge in Online Reputation Management or prior experience in a product-driven organization is a plus.


Additional Skills & Qualities:

A deep understanding of AI/ML lifecycle, from data collection to model deployment and maintenance.
A collaborative mindset, with the ability to work across departments and teams to achieve business objectives.
A passion for innovation and continuous improvement, with a commitment to staying up-to-date on the latest industry trends.
Strong problem-solving abilities, with an analytical and data-driven approach to decision-making.


If you are a highly skilled and motivated data scientist with a passion for cutting-edge technologies and driving impactful business solutions, we encourage you to apply and join our dynamic team at Awone DataScience.



To Apply:

Please submit your resume along with a cover letter outlining your relevant experience and why you would be a great fit for the role on hr@awone.ai



Compensation

Estimated Pay Range: Exact compensation and offers of employment are dependent on circumstances of each case and will be determined based on job-related knowledge, skills, experience, licenses or certifications, and location.
"""

In [23]:
# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

flow.kickoff()

Converting resume to markdown
Resume converted to markdown

Analyzing resume


d:\TechStuff\ai-garage\.venv\Lib\site-packages\crewai\tools\tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"


Resume analysis complete

Saving resume analysis
Resume analysis saved at sample_resume/JonathanWhitmore_DataScience/resume_analysis.json

Aligning resume


d:\TechStuff\ai-garage\.venv\Lib\site-packages\crewai\tools\tool_usage.py:162: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  acceptable_args = tool.args_schema.schema()["properties"].keys()  # type: ignore # Item "None" of "type[BaseModel] | None" has no attribute "schema"


Resume aligned

Saving aligned resume
Aligned resume saved at sample_resume/JonathanWhitmore_DataScience/resume_aligned.md



In [26]:
flow.state.model_dump()

{'job_description': "\nJob Title : Lead Data Scientist\n\nLocation : Hyderabad\n\nCompany : Awone DataScience\n\n\n\nRole Description:\n\nAwone DataScience is seeking an experienced Lead Data Scientist to join our team. This is a full-time on-site opportunity designed for a senior-level professional with expertise in machine learning, AI, and advanced data science techniques. The ideal candidate will be responsible for developing cutting-edge data models, mentoring junior data scientists, and driving the integration of data science solutions into the company’s products. In this role, you will collaborate with cross-functional teams and communicate insights to both technical and non-technical stakeholders, ensuring that data-driven strategies align with the company's objectives.\n\n\n\nKey Responsibilities:\n\nDesign and develop custom data models and algorithms tailored to complex data sets.\nLead the development and implementation of advanced machine learning models to solve intricate